In [1]:
llm_config = {"model": "llama3",
              "base_url": "http://localhost:11434/v1",
              "api_key": "ollama"}

In [2]:
from autogen import AssistantAgent

In [9]:
task = '''
    Write a comprehensive blogspot about summary of 
    Surah Al-baqarah. Make sure the blogspot is within
    1000 words
'''

### Create Writer Agent

In [10]:
writer = AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
    "blogpost (with title) on given topics. You must polish your "
    "writing based on the feedback you receive and give a refined "
    "version. Only return your final work without additional comments.",
    llm_config=llm_config
)

In [11]:
reply = writer.generate_reply(messages=[{"content": task, "role":"user"}])

[autogen.oai.client: 06-18 21:08:26] {294} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


In [12]:
print(reply)

**Unveiling the Wonders of Surah Al-Baqarah: A Comprehensive Summary**

Surah Al-Baqarah, the second chapter of the Quran, is a treasure trove of divine guidance, wisdom, and spiritual nourishment. Spanning 286 verses, this surah is a masterpiece of Islamic literature, offering insights into various aspects of faith, morality, and human nature. In this blog post, we will delve into the essence of Surah Al-Baqarah, exploring its key themes, motifs, and teachings that have guided Muslims for centuries.

**The Structure of Surah Al-Baqarah**

Before diving into the surah's content, it is essential to understand its structure. Surah Al-Baqarah can be divided into three main sections:

1. **Verses 1-44:** These verses focus on the story of Prophet Musa (Moses) and his brother Harun (Aaron), highlighting their struggles and ultimate triumphs.
2. **Verses 45-141:** This section deals with various moral and social issues, such as prayer, charity, and the importance of following God's commandme

### Adding Reflection

In [13]:
critic = AssistantAgent(
    name="Critic",
    system_message="You are a critic. You provide feedback on the "
    "writing style, structure, and content of the blogpost. You must "
    "provide constructive feedback to the writer.",
    llm_config=llm_config,
    is_termination_msg=lambda msg: msg.get("content", "").find("TERMINATE") >= 0
)

In [14]:
response = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


    Write a comprehensive blogspot about summary of 
    Surah Al-baqarah. Make sure the blogspot is within
    1000 words


--------------------------------------------------------------------------------
Writer (to Critic):

**Unveiling the Wonders of Surah Al-Baqarah: A Comprehensive Summary**

Surah Al-Baqarah, the second chapter of the Quran, is a treasure trove of divine guidance, wisdom, and spiritual nourishment. Spanning 286 verses, this surah is a masterpiece of Islamic literature, offering insights into various aspects of faith, morality, and human nature. In this blog post, we will delve into the essence of Surah Al-Baqarah, exploring its key themes, motifs, and teachings that have guided Muslims for centuries.

**The Structure of Surah Al-Baqarah**

Before diving into the surah's content, it is essential to understand its structure. Surah Al-Baqarah can be divided into three main sections:

1. **Verses 1-44:** These verses focus on the story of Prophe

### Nested Chat

In [17]:
SEO_reviewer = AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
    "your ability to optimize content for search engines, "
    "ensuring that it ranks well and attracts organic traffic. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role.",
)

legal_reviewer = AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
    "your ability to ensure that content is legally compliant "
    "and free from any potential legal issues. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role.",
)

ethics_reviewer = AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
    "your ability to ensure that content is ethically sound "
    "and free from any potential ethical issues. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role. ",
)

meta_reviewer = AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [18]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''


review_chats = [
    {
        "recipient": SEO_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt":
                         "Return review into as JSON object only:"
                         "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role", },
        "max_turns": 1},
    {
        "recipient": legal_reviewer, "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt":
                         "Return review into as JSON object only:"
                         "{'Reviewer': '', 'Review': ''}.", },
        "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt":
                      "Return review into as JSON object only:"
                      "{'reviewer': '', 'review': ''}", },
     "max_turns": 1},
    {"recipient": meta_reviewer,
     "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]

In [19]:
critic.register_nested_chats(
    review_chats,
    trigger=writer
)

In [20]:
response2 = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


    Write a comprehensive blogspot about summary of 
    Surah Al-baqarah. Make sure the blogspot is within
    1000 words


--------------------------------------------------------------------------------
Writer (to Critic):

**Unveiling the Wonders of Surah Al-Baqarah: A Comprehensive Summary**

Surah Al-Baqarah, the second chapter of the Quran, is a treasure trove of divine guidance, wisdom, and spiritual nourishment. Spanning 286 verses, this surah is a masterpiece of Islamic literature, offering insights into various aspects of faith, morality, and human nature. In this blog post, we will delve into the essence of Surah Al-Baqarah, exploring its key themes, motifs, and teachings that have guided Muslims for centuries.

**The Structure of Surah Al-Baqarah**

Before diving into the surah's content, it is essential to understand its structure. Surah Al-Baqarah can be divided into three main sections:

1. **Verses 1-44:** These verses focus on the story of Prophe

In [22]:
print(response2.chat_history[-1]['content'])

Unveiling the Wonders of Surah Al-Baqarah: A Comprehensive Summary

Surah Al-Baqarah, the second chapter of the Quran, is a treasure trove of divine guidance, wisdom, and spiritual nourishment. Spanning 286 verses, this surah is a masterpiece of Islamic literature, offering insights into various aspects of faith, morality, and human nature.

**The Structure of Surah Al-Baqarah**

Before diving into the surah's content, it is essential to understand its structure. Surah Al-Baqarah can be divided into three main sections:

1. **Verses 1-44:** These verses focus on the story of Prophet Musa (Moses) and his brother Harun (Aaron), highlighting their struggles and ultimate triumphs.
2. **Verses 45-141:** This section deals with various moral and social issues, such as prayer, charity, and the importance of following God's commandments.
3. **Verses 142-286:** The final portion of the surah addresses specific groups, including the People of the Book (Jews and Christians), and provides guidance

In [23]:
print(response2.summary)

Unveiling the Wonders of Surah Al-Baqarah: A Comprehensive Summary

Surah Al-Baqarah, the second chapter of the Quran, is a treasure trove of divine guidance, wisdom, and spiritual nourishment. Spanning 286 verses, this surah is a masterpiece of Islamic literature, offering insights into various aspects of faith, morality, and human nature.

**The Structure of Surah Al-Baqarah**

Before diving into the surah's content, it is essential to understand its structure. Surah Al-Baqarah can be divided into three main sections:

1. **Verses 1-44:** These verses focus on the story of Prophet Musa (Moses) and his brother Harun (Aaron), highlighting their struggles and ultimate triumphs.
2. **Verses 45-141:** This section deals with various moral and social issues, such as prayer, charity, and the importance of following God's commandments.
3. **Verses 142-286:** The final portion of the surah addresses specific groups, including the People of the Book (Jews and Christians), and provides guidance